# Combatting the Datetime Blues in Python
> Opinionated wrangling of datetime objects



At some point in their careers, all data scientists have suffered from a case of what I like to call "The Datetime Blues."  They come when you least expect. You've worked hard to scrape data from somewhere or query it from some loosely maintained internal database and you feel ready to go. You're super excited to dig into the data, start making some visualizations, forming/testing hypotheses, and get some quick results back to your stakeholders about a project's scope.  However, after importing the data you realize its stored in one of the time formats that you're not 100% familiar with. Ok fine, a quick stackoverflow search and the dataset is ready...right?  Well, not exactly. You then realize that all your metadata is a different time format. Ok, a few more stackoverflow searches and you're up and running again.  "WHAT NO?! Why did this python package add an extra 56 minutes when converting from local time to UTC?!?"  Yes my friend, you have a case of the datetime blues.

Fortunately this only occurs every so often in my work. But it happens just frequently enough to be annoying and just infrequently enough that I forget all the stackoverflow insights I learned the last time I dealt with the problems.  Thus, for my own sake if not yours, I want to put in writing some of the little hiccups and solutions I've come across while working with datetime objects in python and pandas.  I'll probably be adding to this piece so please provide any feedback in the comments. :)

# Opinion: "Time since epoch" vs "Datetime"

Before getting into the code, I would like to mention that, in my experience, a reasonable first step in any time series analysis project is to immediately convert everything into seconds (or milli-/microseconds) since epoch. (Note that by definition, epoch is: `00:00:00 UTC Jan 1, 1970`.) There are several reasons for this but the ones that I find most pertinent are as follows:

- an `int` object is much smaller than a `datetime` or a `str` object. This leads to all sorts of benefits such as smaller storage footprint, faster querying, simpler computation, etc.
- epoch time is timezone independent by its nature. There is no need to ever track down a missing bit of timezone information because it's unclear if it's omission implies UTC or simply a lapse of documentation
- if you work with machine learning models, they understand `int`s but not `datetime` objects

Thus, I typically work with epoch time unless I have some reason to use/see the exact date and time (e.g. visualizations, sanity checks, etc).  However, it's not always clear on how to translate between epoch time and datetime.  To further complicate matters, there are some projects that may require multiple transformations back and forth between the two formats.

# Working with single timestamps

The following will work with converting back and forth between epoch time and datetime for single timestamps.

## Initiating a timezone-aware `datetime`

Despite the above reasons, my human brain still understands datetimes better and, thus, when initiating a timestamp or datetime object (say, for the sake of toy demonstrations for a blog post), I typically start with a datetime representation. However, if I use a datetime object, it is important to make sure that it is "timezone-aware." One of the more ...ahem, enjoyable... aspects of working with datetimes in python is that there are multiple packages to choose from and they all have their own strengths, limitations, and strange nuisances to watch out for. For example, I use `datetime` as the main workhorse of my analysis but I use `pytz` to handle the timezone information of those `datetime` objects.  By way of example, let's initiate an unaware datetime object for `9/13/2019` at `8:10:25am`.

In [2]:
import datetime

unaware_dt = datetime.datetime(2019,9,13,8,10,25)
print('unaware_dt:   ', unaware_dt)

unaware_dt:    2019-09-13 08:10:25


Now, we can try and use the same function call to make a tz-aware datetime object by simply feeding a timezone into the `tzinfo` argument. But... why does the timezone correction piece read: `-04:56` instead of `-04:00`?! To be honest, I remember reading into it at some point and finding the answer too convoluted to care about. Thus, I've decided that the fast-build answer to this is simply: don't initiate tz-aware datetimes like this.

In [3]:
import pytz

nyc_tz = pytz.timezone('US/Eastern')
bad_aware_dt = datetime.datetime(2019,9,13,8,10,25, tzinfo=nyc_tz)

print('unaware_dt:   ', unaware_dt)
print('bad_aware_dt: ', bad_aware_dt)

unaware_dt:    2019-09-13 08:10:25
bad_aware_dt:  2019-09-13 08:10:25-04:56


So how do we initiate a tz-aware datetime object?  Well, one way to do this is to use `pytz`'s `localize` method.  Here you call `localize` on a `pytz` timezone object and pass in an unaware datetime object.  I've found this to be a reliable way to format a tz-aware datetime from scratch.

In [4]:
def mk_aware(dt, tz = pytz.timezone('US/Eastern')): 
    return tz.localize(dt)

good_aware_dt = mk_aware(unaware_dt)

print('unaware_dt:   ', unaware_dt)
print('bad_aware_dt: ', bad_aware_dt)
print('good_aware_dt:', good_aware_dt)

unaware_dt:    2019-09-13 08:10:25
bad_aware_dt:  2019-09-13 08:10:25-04:56
good_aware_dt: 2019-09-13 08:10:25-04:00


And of course, if you have to choose a timezone and you didn't have a preference, I recommend using UTC. Which is done using the above function in the following way:

In [5]:
utc_aware_dt = mk_aware(unaware_dt, tz = pytz.utc)

print('unaware_dt:   ', unaware_dt)
print('utc_aware_dt: ', utc_aware_dt)

unaware_dt:    2019-09-13 08:10:25
utc_aware_dt:  2019-09-13 08:10:25+00:00


## Converting a datetime object to epoch time

In order to convert a `datetime` object to epoch time. the `timestamp` method can be used along with the built in `int()` python function. However, this will always give the results in terms of seconds so if you're interested in milliseconds, simply multiply by 1000.  The only "gotcha" to mention here is that you need to provide a tz-aware `datetime` in order to ensure that the proper time is referenced.

In [12]:
def dt_to_s(dt): return int(dt.timestamp())
def dt_to_ms(dt): return int(dt.timestamp() * 1000)

dt_example = mk_aware(datetime.datetime(2019,9,13,8,10,25),tz=nyc_tz)
print(dt_example)
sec_epoch = dt_to_s(dt_example)
print(sec_epoch)
msec_epoch = dt_to_ms(dt_example)
print(msec_epoch)

2019-09-13 08:10:25-04:00
1568376625
1568376625000


## Converting Epoch time back to Datetime
Again, the name of the game is making the datetime object tz-aware. Note that this is a naive timestamp but it has assumed the timezone to be the local one... so this is kind of dangerous since it gives the local time the timestamp refers to but doesn't tell you that it's doing that.

In [13]:
back2dt = datetime.fromtimestamp(sec_epoch)
print(back2dt)
back2dt = mk_aware(back2dt,tz = nyc_tz)
print(back2dt)

2019-09-13 08:10:25
2019-09-13 08:10:25-04:00


## Converting tz information

I suggest not doing this while working with `datetime` objects.  The following shows that using the `replace` method for `datetime` objects has unpredictable behavior. The conversion from nyc to utc worked well but converting back did not. Again, I always work in epoch time in order to avoid this. (Note: apparently other packages like pendulum and django do not have this problem so feel free to explore those.)

In [21]:
now_utc_dt = good_aware_dt.replace(tzinfo=pytz.utc)
print(now_utc_dt)
now_nyc_dt = now_utc_dt.replace(tzinfo=nyc_tz)
print(now_nyc_dt)

2019-09-13 08:10:25+00:00
2019-09-13 08:10:25-04:56


## Converting from a string

There are several ways to do this. But the one that I like to use is `dateutil.parser` which seems to do everything for me behind the scenes.  Yes, it's another datetime package in python (woohoo).

In [20]:
from dateutil import parser
string = '04/26/2020 8:10am'
dt = parser.parse(string)
dt

datetime.datetime(2020, 4, 26, 8, 10)

# Working with datetime in pandas
Typically, when working with time series data, it will come ina in tabular form.  Let's start by creating a dataframe of 10 minutes of epoch times and some fake data.

In [59]:
import pandas as pd, numpy as np

# make utc aware datetimes for start and end (10 minutes later)
dt_start_time = mk_aware(datetime.datetime(2019,9,13,8,10,25))
dt_end_time = dt_start_time + datetime.timedelta(minutes=10)

# convert to epoch time
start_time = dt_to_s(dt_start_time)
end_time = dt_to_s(dt_end_time)
print(f'From start: {start_time} to end: {end_time}')

# create fake data and time array
time_sec = list(range(start_time,end_time))
fake_data = np.random.normal(size = (len(time_sec),2))

# store in dataframe
df = pd.DataFrame(fake_data, index = time_sec,columns=['col1','col2'])

# reformat dataframe so it's presentable
df.index.name = 'time_sec'
df = df.reset_index()
df.head()

From start: 1568376625 to end: 1568377225


,time_sec,col1,col2
0,1568376625,-1.347006,-1.672383
1,1568376626,-0.259848,1.248307
2,1568376627,-0.964758,-0.412814
3,1568376628,0.066694,-0.014838
4,1568376629,0.019538,-0.154513


## Converting Epoch time to datetime

Pandas has a built in function `pd.to_datetime()` which accepts a numeric column and outputs a datetime column.  Note that we've specified utc as our desired timezone in order to avoid future confusion.

In [61]:
df['utc'] = pd.to_datetime(df.time_sec, unit='s',utc=True)
df.head()

,time_sec,col1,col2,utc
0,1568376625,-1.347006,-1.672383,2019-09-13 12:10:25+00:00
1,1568376626,-0.259848,1.248307,2019-09-13 12:10:26+00:00
2,1568376627,-0.964758,-0.412814,2019-09-13 12:10:27+00:00
3,1568376628,0.066694,-0.014838,2019-09-13 12:10:28+00:00
4,1568376629,0.019538,-0.154513,2019-09-13 12:10:29+00:00


### Converting between timezones
Note that pandas does not suffer from the same issue that datetime does. It is able to repeatably switch between timezones.  Note that in order to operate on the datetime column, we need to access it by using the `dt` accessor.

In [66]:
df['nyc'] = df['utc'].dt.tz_convert(nyc_tz)
df.head()

,time_sec,col1,col2,utc,nyc
0,1568376625,-1.347006,-1.672383,2019-09-13 12:10:25+00:00,2019-09-13 08:10:25-04:00
1,1568376626,-0.259848,1.248307,2019-09-13 12:10:26+00:00,2019-09-13 08:10:26-04:00
2,1568376627,-0.964758,-0.412814,2019-09-13 12:10:27+00:00,2019-09-13 08:10:27-04:00
3,1568376628,0.066694,-0.014838,2019-09-13 12:10:28+00:00,2019-09-13 08:10:28-04:00
4,1568376629,0.019538,-0.154513,2019-09-13 12:10:29+00:00,2019-09-13 08:10:29-04:00


## Converting from `datetime` Column to Epoch Time Column

So this one is a bit tricky.  The primary function that converts from `datetime` to epoch time is just to tell pandas that you want to change the type of the column to `int`.  However, this returns nanoseconds so we need to then account for that by dividing by 1e-9 (which in turn pandas converts to a `float` and then displays with scientific notation). Thus, if you're ultimate goal is a column with type `int` of seconds, you would need to cast it back to `int` again after this rescaling.

In [68]:
df['back2sec'] = df.nyc.astype(int)/1e9
df.head()

,time_sec,col1,col2,utc,nyc,back2sec
0,1568376625,-1.347006,-1.672383,2019-09-13 12:10:25+00:00,2019-09-13 08:10:25-04:00,1.568377e+09
1,1568376626,-0.259848,1.248307,2019-09-13 12:10:26+00:00,2019-09-13 08:10:26-04:00,1.568377e+09
2,1568376627,-0.964758,-0.412814,2019-09-13 12:10:27+00:00,2019-09-13 08:10:27-04:00,1.568377e+09
3,1568376628,0.066694,-0.014838,2019-09-13 12:10:28+00:00,2019-09-13 08:10:28-04:00,1.568377e+09
4,1568376629,0.019538,-0.154513,2019-09-13 12:10:29+00:00,2019-09-13 08:10:29-04:00,1.568377e+09


I created this function just in case I was interested in converting the whole column to the nearest `int`.  I demonstrate that pandas is able to properly handle different timezones by creating two new columns with the function starting from two different timezone columns (utc and nyc).

In [69]:
def series_to_epoch(series,units ='s',return_int=True):
    # choose conversion factor
    if units == 's': conversion = 1e9
    elif units =='ms': conversion = 1e6
    elif units =='us': conversion = 1e3
    
    if return_int:
        return (series.astype(int)/conversion).astype(int)
    else: # returns floats (scientific notation)
        return series.astype(int)/conversion
    
df['back2sec'] = series_to_epoch(df.nyc)
df['back2ms'] = series_to_epoch(df.utc, units='ms')
df.head()

,time_sec,col1,col2,utc,nyc,back2sec,back2ms
0,1568376625,-1.347006,-1.672383,2019-09-13 12:10:25+00:00,2019-09-13 08:10:25-04:00,1568376625,1568376625000
1,1568376626,-0.259848,1.248307,2019-09-13 12:10:26+00:00,2019-09-13 08:10:26-04:00,1568376626,1568376626000
2,1568376627,-0.964758,-0.412814,2019-09-13 12:10:27+00:00,2019-09-13 08:10:27-04:00,1568376627,1568376627000
3,1568376628,0.066694,-0.014838,2019-09-13 12:10:28+00:00,2019-09-13 08:10:28-04:00,1568376628,1568376628000
4,1568376629,0.019538,-0.154513,2019-09-13 12:10:29+00:00,2019-09-13 08:10:29-04:00,1568376629,1568376629000
